In [32]:
import pandas as pd
import numpy as np 
from typing import List,Tuple
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# operating base for our nerual network

In [170]:
def assert_same_shape(array:np.ndarray,
                     array_grad:np.ndarray)->None:
    assert array.shape == array_grad.shape,\
        '''the shape of the nd array should be the same 
          but the shape of the array's are {0} for the first one and
          the second array shape are {1}'''.format(tuple(array.shape),tuple(array_grad.shape))
    return None

In [171]:
class operation(object):
    def __init__(self):
        pass
    def forward(self,input_:np.ndarray):
        self.input_=input_
        self.output=self._output()
        return self.output
    def backward(self,output_grad:np.ndarray)->np.ndarray:
        assert_same_shape(self.output,output_grad)
        self.input_grad=self._input_grad(output_grad)
        assert_same_shape(self.input_,self.input_grad)
        return self.input_grad
    def _output(self)->np.ndarray:
        raise NotImplementedError()
    def _input_grad(self,output_grad:np.ndarray)->np.ndarray:
        raise NotImplementedError

In [172]:
class paramoperation(operation):
    def __init__(self,param:np.ndarray):
        super().__init__()
        self.param=param
        
    def backward(self,output_grad:np.ndarray):
        assert_same_shape(self.output,output_grad)
        self.input_grad=self._input_grad(output_grad)
        self.param_grad=self._param_grad(output_grad)
        #we have to check for the shapes of the variables
        assert_same_shape(self.input_,self.input_grad)
        assert_same_shape(self.param,self.param_grad)
        return self.input_grad
        
    def _param_grad(output_grad:np.ndarray):
        raise NotImplementedError()

In [173]:
class weightmultiply(paramoperation):
    def __init__(self,w:np.ndarray):
        super().__init__(w)  #this is going to be send to the parent class
    def _output(self)->np.ndarray:
        return np.dot(self.input_,self.param)  #this is the first forward of the nerual network
        
    def _input_grad(self,output_grad:np.ndarray)->np.ndarray:
        ''' we are dervating with respect with the input'''
        return np.dot(output_grad,np.transpose(self.param,(1,0)))
         
    def _param_grad(self,output_grad:np.ndarray)->np.ndarray:
        '''we are derivating with respect to the parameters or the weights '''
        return np.dot(np.transpose(self.input_,(1,0)),output_grad)
        

In [174]:
class biasadd(paramoperation):
    
    def __init__(self,bias):
        assert bias.shape[0]==1
        super().__init__(bias)
        
    def _output(self):
        ''' the other values is being treated as bias'''
        return self.input_ + self.param
        
    def _input_grad(self,output_grad:np.ndarray)->np.ndarray:
        ''' this is with respect with the bias '''
        return np.ones_like(self.input_)*output_grad
        
    def _param_grad(self,output_grad:np.ndarray)->np.ndarray:
        '''this derivation is with respect to the bias'''
        param_grad= np.ones_like(self.param)*output_grad
        return np.sum(param_grad,axis=0).reshape(1,param_grad.shape[1])

In [175]:
class sigmoid(operation):
    '''this is sigmoid function'''
    def __init__(self):
        super().__init__()
        
    def _output(self):
        return 1.0/(1.0+np.exp(-1.0*self.input_))
        
    def _input_grad(self,output_grad:np.ndarray):
        ''' this is the input grad with respect to the sigmoid fucntion'''
        sigmoid_back=self.output*(1-self.output)
        input_grad=sigmoid_back*output_grad
        return input_grad

In [176]:
class linear(operation):
    ''' this is identity activation function for our A.N.N'''
    def __init__(self)->None:
        super().__init__()
    def _output(self)->np.ndarray:
        return self.input_
    def _input_grad(self,output_grad:np.ndarray)->np.ndarray:
        return output_grad

# this is layers of neural network from scratch

In [177]:
class Layer(object):
    
    def __init__(self,neurons:int):
        
        self.neurons=neurons
        self.first=True
        
        self.param:List[np.ndarray]=[]
        self.param_grads:List[np.ndarray]=[]
        
        self.operations:List[operation]=[]
        
    def _setup_layer(self,num_in:int):
        raise NotImplementedEror()
        
    def forward(self,input_:np.ndarray):
        if self.first:
            self._setup_layer(input_)
            self.first=False
        self.input_=input_
        for operation in self.operations:
            input_=operation.forward(input_)
        self.output=input_
        return self.output
        
    def backward(self,output_grad:np.ndarray)->np.ndarray:
        '''lets check the appropriate shape of the output and 
        output grad and perform series of backpropagation'''
        
        assert_same_shape(self.output,output_grad)
        
        for operation in reversed(self.operations):
            output_grad=operation.backward(output_grad)   # the operation is going back ward from output into input
        
        input_grad=output_grad  #because its moving from the far-left of the network to the right
        self._params_grad()
        
        return input_grad
    def _params_grad(self)->np.ndarray:
        ''' this is usef to exrtact the param_grad from layers'''
        self.param_grads=[]
        for operation in self.operations:
            if issubclass(operation.__class__,paramoperation):   #check if the class is a sub class of the "paramoperation" class
                self.params.append(operation.param_grad)
                
    def _params(self)->np.ndarray:
        ''' this is used to extract the param from the layers'''
        self.param=[]
        for operation in self.operations:
            if subclass(operation.__class__,paramoperation):
                self.param.appened(operation.param)
        

# know lets make dense layers

In [178]:
class dense(Layer):
    ''' this is a fully conected layer inherted from layer'''
    
    def __init__(self,
                 neurons:int,
                 activation:operation=sigmoid()
                )->None:
        super().__init__(neurons)
        self.activation=activation
        
    def _setup_layer(self,input_:np.ndarray):
        
        ''' this whole thing defines the function of a connected layer '''
        
        if self.seed:
            np.random.seed(self.seed)
        
        self.params=[]
        
        #weights
        self.params.append(np.random.randn(input_.shape[1],self.neurons))

        #bias
        self.params.append(np.random.randn(1,self.neurons))
        
        #the operation
        self.operations=[weightmultiply(self.params[0]),
                        biasadd(self.params[1]),
                        self.activation]
        return None

# lets make a base loss layer

In [179]:
class Loss(object):
    ''' the loss of a neural network '''
    
    def __init__(self):
        pass
    
    def forward(self,prediction:np.ndarray,target:np.ndarray):
        ''' this computes the loss of the neural net'''
        assert_same_shape(prediction,target)
        self.prediction=prediction
        self.target=target
        loss_value=self._output()
        return loss_value
    
    def backward(self)->np.ndarray:
        ''' this is for calculating the derivative with respect to the inputs'''
        self.input_grad=self._input_grad()
        assert_same_shape(self.prediction,self.input_grad)  #this is for checking the rule that said the input and output grad must be the same
        return self.input_grad
   
    def _output(self)->float:
        raise NotImplementedError()
        
    def _input_grad(self):
        raise NotImplementedError()

In [180]:
class meansquarederror(Loss):
    def __init__(self)->None:
        super().__init__()
    def _output(self)->float:
        ''' this computes the mean squred error for the predicted and the truth'''
        loss=(
            np.sum(np.power(self.prediction - self.target,2))/self.prediction.shape[0]   # we are dividing  by the number of predictions to get mean squared error 
        )
        return loss
    def _input_grad(self)->np.ndarray:
        ''' this calculated the gradient with respect to the input..in this case
        the input is the predicted value
        '''
        return 2.0*(self.prediction - self.target)/self.prediction.shape[0]
    

# lets build a fully connected neural network

In [181]:
class neuralnetwork(object):
    ''' we are going to make a neural network form all the operations,
    layers and loss classes we have made so far '''
    
    def __init__(self,
                layers:List[Layer],
                loss:Loss,
                seed: float=1):
        
        self.layers=layers
        self.loss=loss
        self.seed=seed
        #lets make all the layers have the same seed number 
        if seed:
            for layer in self.layers: 
                setattr(layer,"seed",self.seed)
                
    #this is the forward pass of the neural network
    def forward(self,x_batch:np.ndarray):
        x_out=x_batch
        for layer in self.layers:
            x_out=layer.forward(x_out)
        return x_out
            
        #this is the backward pass of the nerual network
    def backward(self,loss_grad:np.ndarray)->None:
        grad=loss_grad  #this grad is the last layers grad
        for layer in reversed(self.layers):
            grad=layer.backward(grad)
        return None
        
        #this computes the "training" part of the nerural network
    def train_batch(self,x_batch,y_batch)->float:
        prediction = self.forward(x_batch)
        loss=self.loss.forward(prediction,y_batch)
        self.backward(self.loss.backward())  #this is basically the backpropagation
        return loss
    def params(self):
        ''' this is for retriving the parameters of the neural network'''
        
        for layer in self.layers:
            
            yield from layer.params
    def param_grads(self):
        ''' this is for retreiving the parameter gradient'''
        for layer in self.layers:
             yield from layer.param_grads
                

# lets build optimizer and trainer

## optimizer

In [182]:
class optimizer(object):
    ''' this is for optimizing the weights of the neurla network '''
    def __init__(self,lr:float=0.4):
        '''we use learning rate specifying the rate at which the model is going to be build'''
        self.lr=lr
    def step(self):
        '''this is for calculating or implmenting of the optimizing'''
        pass

In [183]:
class SGD(optimizer):
    ''' this optimizer is the stochastic gradient decent'''
    def __init__(self,lr:float=0.01)->None:
        super().__init__(lr)
        
    def step(self):
        '''we adjust the parameters based on the learning rate of the model'''
        for (param,param_grad) in zip(self.net.params(),self.net.param_grads()):
            param-=self.lr*param_grad
            

## trainer

In [184]:
def permute_data(x:np.ndarray,y:np.ndarray):
    perm=np.random.permutation(x.shape[0])
    return x[perm],y[perm]

In [185]:
class Trainer(object):
    ''' this trains the neural network '''
    def __init__(self,
                net:neuralnetwork,
                optim:optimizer)->None:
        self.net=net
        self.optim=optim
        self.best_loss=1e9
        setattr(self.optim,"net",self.net)  #the self.net is going to be an attribute of self.optim
        
    def generate_batches(self,
                         x:np.ndarray,
                         y:np.ndarray,
                        size:int=32)->Tuple[np.ndarray]:
        ''' this generates batch for the training'''
        
        assert x.shape[0]==y.shape[0],\
        '''feature and targets must have the number of rows
        but in these case feature is {0} and target is {1}'''.format(x.shape[0],y.shape[0])
        
        n=x.shape[0]  #this is the amount of training data we have 
        for i in range(0,n,size):
            x_batch,y_batch=x[i:i+size],y[i:i+size]
            yield x_batch,y_batch

    
    def fit(self,x_train:np.ndarray,y_train:np.ndarray,
           x_test:np.ndarray,y_test:np.ndarray,
           epochs:int=100,eval_every:int=32,batch_sz:int=32,seed:int=32,restart:bool=True)->None:
        ''' this trains the neural network '''
        np.random.seed(seed)
        if restart:
            ''' this restart the weights or the parameters to some initial values'''
            for layer in self.net.layers:
                layer.first=True
            self.best_loss=1e9
            
        for i in range(epochs):
            #this loop goes on unil the training is Done
            if (i+1)%eval_every == 0:
                #this is sort of early stoping
                last_model=deepcopy(self.net)
                
            x_train,y_train=permute_data(x_train,y_train)  #this shuffles the data
            batch_generator=self.generate_batches(x_train,y_train,batch_sz)
            for j,(x_batch,y_batch) in enumerate(batch_generator):
                self.net.train_batch(x_batch,y_batch)
                self.optim.step()
                
            if (i+1)%eval_every==0:
                #this check for what it had learned every some terms on the test set
                test_preds=self.net.forward(x_test)
                loss=self.net.loss.forward(test_preds,y_test)
                #this is basically Early stoping
                if loss < self.best_loss:
                    print(f"validation(test) loss after {i+1} epochs is {loss:.3f}")
                    self.best_loss=loss
                else:
                    print(f"loss increased after {i+1} epochs,final good loss is {self.best_loss:.3f}")
                    self.net=last_model
                    setattr(self.optim,"net",self.net)
                    break
                    

## error evaluation metrics

In [186]:
def mae(y_true:np.ndarray,y_pred:np.ndarray):
    ''' performing the MAE for neurla network'''
    return np.mean(np.abs(y_true-y_pred))

In [187]:
def rmse(y_true:np.ndarray,y_pred:np.ndarray):
    ''' this performs the RMSE'''
    return np.sqrt(np.mean(np.power(y_true-y_pred,2)))

In [188]:
def eval_regression_model(model:neuralnetwork,
                          x_test:np.ndarray,
                          y_test:np.ndarray):
    ''' this computes the mae and rmse '''
    preds=model.forward(x_test)
    preds=preds.reshape(-1,1)
    print(" the mean absolute error is: {:.2f}".format(mae(y_test,preds)))
    print() #empty line
    print("root mean squred error:{:.2f}".format(rmse(y_test,preds)))

# lets train our deep neural network that we just created YAY

In [189]:
#lets test how our deep neural network
lr=neuralnetwork(
    layers=[dense(neurons=1,
                 activation=linear())],
    loss=meansquarederror(),
    seed=42
)
nn=neuralnetwork(
    layers=[
        dense(neurons=13,
           activation=sigmoid()),
        dense(neurons=1,
             activation=linear())
    ],
    loss=meansquarederror(),
    seed=42
)
dl=neuralnetwork(
    layers=[
        dense(neurons=13,activation=sigmoid()),
        dense(neurons=13,activation=sigmoid()),
        dense(neurons=13,activation=linear())
    ],
    loss=meansquarederror(),
    seed=42
)

## lets some data and do some EDA

In [159]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [160]:
s=StandardScaler()
data=s.fit_transform(data)

In [161]:
x_train,x_test,y_train,y_test=train_test_split(data,target,random_state=42)

In [193]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

### lets train one of our 3 model 

In [190]:
trainer=Trainer(lr,SGD(lr=0.01))
trainer.fit(x_train,y_train,x_test,y_test,epochs=100,
           eval_every=10,seed=42)
print()
eval_regression_model(lr,x_test,y_test)

validation(test) loss after 10 epochs is 652.977
loss increased after 20 epochs,final good loss is 652.977

 the mean absolute error is: 23.50

root mean squred error:25.55


In [196]:
trainer=Trainer(nn,SGD(lr=0.01))
trainer.fit(x_train,y_train,x_test,y_test,
           eval_every=10,seed=42)
print()
eval_regression_model(nn,x_test,y_test)

validation(test) loss after 10 epochs is 551.293
loss increased after 20 epochs,final good loss is 551.293

 the mean absolute error is: 21.99

root mean squred error:23.48


In [ ]:
# THE END 